# INDUS Downstream Fine-Tuning: Climate Indicator Classification


### Overview
This notebook focuses on finetuning the INDUS Encoder model for various Downstream tasks.

- Learn how to use Huggingface to finetune INDUS family of models.
- Understand the effects of spefic parameters in finetuning.
- Use finetuned model for inference.
- Understand the unique aspects of encoder models for Downstream tasks.

In [1]:
%pip install pandas accelerate transformers torch scikit-learn marvin openpyxl


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Setup
Go to "Kernel"
Select "prithvi_eo"### Load data

In [2]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
## encode labels
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset

/Users/mramasub/work/ESA-NASA-workshop-2025/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
excel_file_path = "ej_dataset.xlsx"
df = pd.read_excel(excel_file_path, engine="openpyxl")

df = df.dropna()
df.sample(frac=1).reset_index(drop=True)

,Indicators,Description
0,Human Dimensions,The Global Cyclone Total Economic Loss Risk De...
1,Human Dimensions,The Global Multihazard Proportional Economic L...
2,Extreme Heat,The Global High Resolution Daily Extreme Urban...
3,Climate Change,This dataset provides daily-averaged global me...
4,Human Dimensions,The Global One-Eighth Degree Urban Land Extent...
...,...,...
247,Human Dimensions,The Global 1-km Downscaled Urban Land Extent P...
248,Human Dimensions,The Global Development Potential Indices (DPIs...
249,Human Dimensions,The Global Flood Total Economic Loss Risk Deci...
250,Urban Flooding,NASADEM data products were derived from origin...


## Load the Encoder Model using Huggingface Lib

In [4]:

# Load your domain-specific encoder model (replace 'model_name' with your model's name)
model_name = 'nasa-impact/nasa-smd-ibm-distil-v0.1'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=8)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at nasa-impact/nasa-smd-ibm-distil-v0.1 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Encode the data and Labels into ML-ready format

In [5]:
# Create a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }

label_encoder = LabelEncoder()
df['labels'] = label_encoder.fit_transform(df['Indicators'])
# print the label mapping
for index, item in enumerate(label_encoder.classes_):
    print(item, '->', index)

data = df
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42, stratify=data['labels'])
train_encodings = tokenizer(list(train_data['Description']), return_tensors='pt', padding=True, truncation=True, max_length=512, return_attention_mask=True)
test_encodings = tokenizer(list(test_data['Description']), return_tensors='pt', padding=True, truncation=True, max_length=512, return_attention_mask=True)
train_dataset = CustomDataset(train_encodings, torch.tensor(train_data['labels'].tolist()))
test_dataset = CustomDataset(test_encodings, torch.tensor(test_data['labels'].tolist()))


Climate Change -> 0
Disasters -> 1
Extreme Heat -> 2
Food Availability -> 3
Health & Air Quality -> 4
Human Dimensions -> 5
Urban Flooding -> 6
Water Availability -> 7


## Create training and evaluation arguments for the Huggingface Trainer

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    # evaluation_strategy="epoch",
    output_dir="./output",
    num_train_epochs=10,
    save_steps=50,
    save_total_limit=2,
    remove_unused_columns=True,
    logging_dir="./logs",
    optim="adamw_torch",
    learning_rate=5e-5,
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=lambda p: classification_report(p.label_ids, p.predictions.argmax(-1), output_dict=True),
)

trainer.train()

results = trainer.evaluate()

# Print classification report
print("Classification Report:")
print(results)

## Evaluation on the test set

In [8]:
results

{'eval_loss': 1.20835280418396,
 'eval_0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 4.0},
 'eval_1': {'precision': 0.5,
  'recall': 0.3333333333333333,
  'f1-score': 0.4,
  'support': 3.0},
 'eval_2': {'precision': 1.0,
  'recall': 0.3333333333333333,
  'f1-score': 0.5,
  'support': 6.0},
 'eval_3': {'precision': 1.0,
  'recall': 0.6666666666666666,
  'f1-score': 0.8,
  'support': 3.0},
 'eval_4': {'precision': 0.7222222222222222,
  'recall': 1.0,
  'f1-score': 0.8387096774193549,
  'support': 13.0},
 'eval_5': {'precision': 0.7619047619047619,
  'recall': 0.9411764705882353,
  'f1-score': 0.8421052631578947,
  'support': 17.0},
 'eval_6': {'precision': 1.0,
  'recall': 0.5,
  'f1-score': 0.6666666666666666,
  'support': 4.0},
 'eval_7': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1.0},
 'eval_accuracy': 0.7843137254901961,
 'eval_macro avg': {'precision': 0.748015873015873,
  'recall': 0.5968137254901961,
  'f1-score': 0.6309352009054896,
  'supp

## save model for later use

In [ ]:

# Save the trained model
model.save_pretrained('ej_classifier')  # Replace 'your_model_directory' with your desired directory
tokenizer.save_pretrained('ej_tokenizer')  # Save the tokenizer as well

('ej_tokenizer/tokenizer_config.json',
 'ej_tokenizer/special_tokens_map.json',
 'ej_tokenizer/vocab.txt',
 'ej_tokenizer/added_tokens.json',
 'ej_tokenizer/tokenizer.json')

## Predict using the saved model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder
import torch
import pandas as pd

# Load the saved model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained('ej_classifier')  # Load from the directory where you saved it
tokenizer = AutoTokenizer.from_pretrained('ej_tokenizer')

# Get predictions
with torch.no_grad():
    outputs = model(**test_encodings)

# Extract predicted class labels
predicted_labels = torch.argmax(outputs.logits, dim=1)

predicted_class_labels = label_encoder.inverse_transform(predicted_labels.numpy())

# print the text, true and predicted labels
for i in range(len(test_data)):
    print(test_data['Description'].iloc[i])
    print('True:', test_data['Indicators'].iloc[i])
    print('Predicted:', label_encoder.classes_[predicted_labels[i]])
    print('')